# Using a Transformer model to predict sequences

A transformer model is a form of deep learning architecture. It is used to predict what should come next given an input sequence. As all other deep learning models, a transformer must also be trained. An attention layer makes sure that information is propagated from earlier in the sequence to the later parts of the sequence. In this way, it is easier for the model to connect all the data from the input sequence and give a better prediction. In this notebook neural networks will be trained to sort a sequence of integers and add together two two-digit non-negative integer numbers. 

---

## 1: Introductory exercises

### 1.1

We want to train a model that predicts $d$ given $d = a \cdot b + c$.
$a$, $b$ and $c$ are non-negative integers. $a$ and $c$ are two-digit integers, while $b$ is a one-digit integer.
This makes $d$ at most a three digit number. Specifically $d \in \{ 0, 1, 2, \dots, 989, 990 \}$. The representation of $d$ then becomes
$n_0 n_1 n_2$.
Because we reverse the digits, the training set $\{x, y\}$ would become:

\begin{align*}
    x &= [ a_0, a_1, b, c_0, c_1, d_2, d_1 ] \\
    y &= [  d_2, d_1, d_0 ]
\end{align*}

A concrete example shows that padding $a$ with a leading zeros keeps the length constant:

$$
    a = 5, b = 5, c = 33 \\
    a \cdot b + c = 58
$$

gives

\begin{align*}
    x &= [0,5,5,3,3,8,5] \\
    y &= [8,5,0].
\end{align*}

### 1.2

When the network is optimized it will predict $d$ given $a$, $b$ and $c$. Using the same $a = 5, b = 5, c = 33$ as before:

\begin{align*}
    x^{(0)} &= [0, 5, 5, 3, 3],  &[\hat{z}_0^{(0)}, \hat{z}_1^{(0)}, \hat{z}_2^{(0)}, \hat{z}_3^{(0)}, \textcolor{red}{8}] = f_\theta(x^{(0)})\\
    x^{(1)} &= [0, 5, 5, 3, 3, \textcolor{red}{8}],  &[\hat{z}_0^{(1)}, \cdots, \textcolor{blue}{5}] = f_\theta(x^{(1)})  \\
    x^{(2)} &= [0, 5, 5, 3, 3,  \textcolor{red}{8}, \textcolor{blue}{5}],  &[\hat{z}_0^{(2)}, \cdots, \textcolor{green}{0}] = f_\theta(x^{(2)}) \\
    x^{(3)} &= [0, 5, 5, 3, 3, \textcolor{red}{8}, \textcolor{blue}{5}, \textcolor{green}{0}] \\
\end{align*}

\begin{align*}
\hat{y} = [\textcolor{red}{8}, \textcolor{blue}{5}, \textcolor{green}{0}]
\end{align*}




### 1.3

Using cross entropy as the object function, with $m = 5$ and $y = [4, 3, 2, 1]$. If the object function $\mathcal{L(\theta, D)} = 0$, then $\hat{Y}$ would be the onehot representation of the correct solution $y$: 
\begin{align*}
\hat{Y} = \text{onehot}(y) =
\begin{bmatrix}
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 \\
0 & 0 & 1 & 0 \\
0 & 1 & 0 & 0 \\
1 & 0 & 0 & 0 \\
\end{bmatrix}
\end{align*}

In this case $ \hat{y} = y = [4, 3, 2, 1]$.
If the object function $\mathcal{L(\theta, D)} = 0$, then $ \hat{y}$ will be the same as the solution $y$.


### 1.4
Given $ d, m, n_{max}, k, p, L$. To find the amount of parameters that must be determined one must look at the dimensions of all the matrices involved in the neural network.

$W_{E},  W_{U} \in \mathbb{R}^{d \times m}$ and $W_{P} \in \mathbb{R}^{d \times n_{max}}$ are only made once per neural network.

$W_{O},  W_{V}, W_{Q},  W_{K} \in \mathbb{R}^{k \times d}$ and $W_{1},  W_{2} \in \mathbb{R}^{p \times d}$ are made for all $L$ layers in the transformer.

In total that gives $ 2 \cdot dm  + dn_{max} + L(4\cdot kd + 2\cdot pd)$ individual parameters that must be determined.
 

### 1.5  
For a neural network with parameters $ W_{O}, W_{V}, W_{Q}, W_{K}, W_{1}, W_{2}, W_{U} = I_{2 \times 2}$,  $\sigma(x) = \text{Relu}(x)=\text{max}(0, x)$  and 
\begin{align*} W_{E} = \begin{bmatrix} 1 & 0 \\ 0 & \alpha \end{bmatrix} \ \text{and} \ W_{P} = \begin{bmatrix} 1 \\ 0 \end{bmatrix} \end{align*}
  Given an input $x=[1]$, $n = n_{max} = 1$, $m = d = k = p = 2$ and $L = 1$, $\alpha$ must be larger than 1 to get $\hat{z} = [1]$ as a prediction, as shown below:

\begin{align}
X &= \text{onehot}(x) = \begin{bmatrix} 0 \\ 1 \end{bmatrix} \\
z_{0} &= W_{E}X + [W_{P}]_{0:n} = \begin{bmatrix}1 & 0 \\ 0 & \alpha \end{bmatrix} \begin{bmatrix}0 \\ 1
\end{bmatrix} + \begin{bmatrix} 1 \\ 0 \end{bmatrix} = \begin{bmatrix} 1\\ \alpha \end{bmatrix} \\

z_{\frac{1}{2}} &= f_{d}^{A}(z_0) = z_0 + W_{O}^{T}W_{V}z_{0}A(z_0), &\ W_{O}^{T}W_{V} = I_{2 \times 2}I_{2 \times 2} = 1 \\
&= z_0 + z_0 \text{softmax}_{\text{col}}(z^{T}W_{Q}^{T}W_{K}z + D), &\ D = [0], \ W_{Q}^{T}W_{K} = I_{2 \times 2}I_{2 \times 2} = 1\\
&= \begin{bmatrix} 1 \\ \alpha \end{bmatrix} + \begin{bmatrix} 1 \\ \alpha \end{bmatrix} \text{softmax}_{\text{col}}(1 + \alpha^2) = \begin{bmatrix}2 \\ 2\alpha \end{bmatrix} \\

z_1 &= f_1^{F}(z_{\frac{1}{2}}) = z_{\frac{1}{2}} + W_2^{T}\sigma(W_1z_{\frac{1}{2}}) = \begin{bmatrix}2 \\ 2\alpha \end{bmatrix} + \sigma\left(\begin{bmatrix} 2 \\ 2\alpha \end{bmatrix}\right) \\ 
&= \begin{bmatrix}2 \\ 2\alpha \end{bmatrix} + \begin{bmatrix}2 \\ 2\alpha \end{bmatrix},&\ \text{given} \ \alpha > 0 \\

Z &= \text{softmax}_{\text{col}}\left(\begin{bmatrix}4 \\ \alpha\end{bmatrix}\right) = \begin{bmatrix} \frac{e^4}{e^4+e^{4\alpha}} \\ \frac{e^{4\alpha}}{e^4+e^{4\alpha}} \end{bmatrix} \\
\hat{z} &= \text{argmax}_{\text{col}}(Z) \\
\end{align}
 
$\text{if} \ \hat{z} = [1] \ \text{then we must have that}$ 
\begin{align*} 
\frac{e^{4\alpha}}{e^4+e^{4\alpha}} &> \frac{e^{4}}{e^4+e^{4\alpha}}\\
e^{4\alpha} &> e^{4} \\
4\alpha &> 4 \\
\alpha &> 1
\end{align*}

This works given $\alpha > 0$. For $\alpha \leq 0$ one must look from eq. (6) where $\sigma = \text{max}(0, x)$

\begin{align*}
z_1 &= f_1^{F}(z_{\frac{1}{2}}) = z_{\frac{1}{2}} + W_2^{T}\sigma(W_1z_{\frac{1}{2}}) = \begin{bmatrix}2 \\ 2\alpha \end{bmatrix} + \sigma\left(\begin{bmatrix} 2 \\ 2\alpha \end{bmatrix}\right) \\
&= \begin{bmatrix}2 \\ 2\alpha \end{bmatrix} + \begin{bmatrix}2 \\ 0 \end{bmatrix} = \begin{bmatrix} 4 \\ 2\alpha \end{bmatrix},&\ \text{given} \ \alpha \leq 0 \\

Z &= \text{softmax}_{\text{col}}\left(\begin{bmatrix} 4 \\ 2\alpha \end{bmatrix}\right) = \begin{bmatrix} \frac{e^4}{e^4+e^{2\alpha}} \\ \frac{e^{2\alpha}}{e^4+e^{2\alpha}} \end{bmatrix} \\
\hat{z} &= \text{argmax}_{\text{col}}\left(Z\right)
\end{align*}

If $\hat{z} = [1]$ we must have

\begin{align*}
\frac{e^{2\alpha}}{e^4+e^{2\alpha}} &> \frac{e^4}{e^4+e^{2\alpha}} \\ 
\end{align*}
But $\nexists \alpha \leq 0 \ \text{s.t.} \ e^{2\alpha} > e^{4} $ and therefore $\alpha > 1$. $\square$

## 2: Implementing the network

### 2.1

To perform a gradient descent step on a `NeuralNetwork`, the member function `step_gd` is called. This function then calls on the `step_gd` method for all the relevant layers in the network. `step_gd` has a default implementation in the class `Layer`, and all subclasses of `Layer`  will inherit this method unless specifically overwritten in the given subclass. All of the layers used in the network are subclasses of `Layer` and thus inherit `step_gd` from it, although layers such as `FeedForward` and `EmbedPosition` overwrite it with a custom implementation of their own. Even though these classes overwrite the inherited method, they can still use it by calling `super().step_gd()` in their own implementation. This is for example done in `EmbedPosition`. `FeedForward` on the other hand uses the `step_gd` of its two sublayers.

### 2.2 and 2.3

We refer to the files `layers.py` and `layers_numba.py`.

### Extras

#### `Numba`

In an effort to speed up our code, we reimplemented all the layers using numba. By doing so we managed to get the most resource intensive part of the training compiled to much faster machine code. The downside is that it spends a lot of time compiling the code before it runs. For long running training sessions, this is worth it because of the speed up. However, for programs running for only about a minute or less, the compilation overhead is too big to save any time, and the normal implementation finishes first. Another note to make is that when running JIT-compiled code, we get some performance warnings from the compiler. These originate from a part of the code that tries to do operations on the columns of three dimensional matrices, thus producing several cache misses. This problem can be solved in different ways, but we found that the current implementation worked the best nonetheless.

#### Dumping and loading

When starting to train bigger models, we found that a way to save and load the trained networks would be beneficial. Storing the original layers, non-numba layers, proved to be really simple with `pickle` or `dill`. The numba-layers on the other hand could not be serialized as is, and for that reason we had to implement some extra utility functions for converting the data into a serializable format to be dumped with `dill`. These functions were implemented in `neural_network.py`.

## 3: Training the nework and solving problems

In order to keep all the parameters (dimensions, length of input/output, number of iterations etc.) at one place we decided to store these in separate dataclasses in `train_test_params.py`. See the comments in `BaseParams` for an explanation of the member variables.

In [ ]:
from train_test_params import SortParams1, SortParams2, AddParams

sort_params1 = SortParams1()    # parameters in part 1 of exercise 3.3
sort_params2 = SortParams2()    # parameters in part 2 of exercise 3.3
add_params = AddParams()

### 3.1

`test_implementation.ipynb` was used to test the implementation of the layers. We had to make a few changes to the tests:

When computing the value of our loss function we have to slice $Z$ such that we only use the columns that are useful.

How to slice is determined by making sure that all $x$'s and $y$'s follow the dimensions given in part 3.2.1 in the project description.

```python
L = loss.forward(Z[:, :, -n_y:], y)
```
In addition we have to take into account that the backward pass of Softmax is defined as

\begin{align*}
\frac{\partial \mathcal{L}}{\partial z} := g_l \odot z_l - \text{sum}_\text{col}(g_l \odot S) \odot P.
\end{align*}

Here $g_l \in \R^{m \times n_y}$ and $z_l \in \R ^{m \times n}$. In order to match the dimensions we left-pad the gradient with zeros:

```python
pad_matrix = np.zeros((b, m, n_max - n_y))
grad_Z = np.concatenate((pad_matrix, grad_Z), axis=2)
```

### 3.2

The file `train_network.py` has two functions:

- `init_neural_network` creates an object of type `NeuralNetwork` given the parameters that the neural network will use.

- `train_network` is the implementation of Algorithm 4 in the assignment. It trains the neural network and uses the Adam algorithm to change the individual parameters of the network for each iteration. We also added support for dumping the network during training.

In order to look at how the loss function develops we plot the result on a logarithmically scaled y-axis. 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_loss(loss: list[np.ndarray], title: str, labels: None|tuple[str] = None) -> None:
    """ Plots the loss as a function of iteration step

    Args:
        loss: list of loss values
        title: title of the plot
        labels: optional labels
    """
    iters = [];L = []
    for l in loss:
        iters.append([i+1 for i in range(len(l[l > 0]))])
        L.append(l[l>0])

    fig, ax = plt.subplots()
    if labels == None:
        for i, l in zip(iters, L):
            ax.plot(i, l)
    else:
        for i, l, lab in zip(iters, L, labels):
            ax.plot(i, l, label=lab)
    ax.set(
        title=title,
        ylabel=r"Loss $\mathcal{L}$",
        xlabel="Iterations",
        yscale="log",
    )
    ax.grid()
    if labels != None: ax.legend()
    plt.show()

### 3.3: Sorting problem

#### Simple sorting of binary sequences

Here we sort sequences only consisting of zeros and ones. A quick example never hurts:

$$
[0,1,1,1,0] \to [0,0,1,1,1]
$$

Prepare training and testing data for the sorting problem described in the first part of exercise 3.3:

In [ ]:
from data_generators import get_train_test_sorting

training_data = get_train_test_sorting(
    length=sort_params1.r,
    num_ints=sort_params1.m,
    samples_per_batch=sort_params1.D,
    n_batches_train=sort_params1.b_train,
    n_batches_test=sort_params1.b_test,
)

x_train = training_data["x_train"]
y_train = training_data["y_train"][:, :, sort_params1.r - 1:]
x_test = training_data["x_test"]
y_test = training_data["y_test"]

Let's initialize the network with r = $ 5$ and m = $2$.

In [ ]:
from train_network import init_neural_network

network = init_neural_network(sort_params1)

Train the network using `CrossEntropy` as the loss function (object function).

If the value of our object function $\mathcal{L}$ gets lower than $0.01$ we stop the training.
Otherwise the training runs for $n_{\text{iter}} = 300$ iterations.

In [ ]:
from train_network import train_network
from layers_numba import CrossEntropy

loss = CrossEntropy()

L = train_network(
    network=network,
    x_train=x_train,
    y_train=y_train,
    loss_func=loss,
    alpha=sort_params1.alpha,
    n_iter=sort_params1.n_iter,
    num_ints=sort_params1.m,
    dump_to_pickle_file=False,
)

In [ ]:
plot_loss([L], "Training for the sorting problem part 1")

We can also load a pre-trained network from a pickle dump as explained in part 3.2.

In [ ]:
loaded_network = init_neural_network(sort_params1)
loaded_network.load("SUPPLY_FILENAME_HERE")            # this will override all parameters initialized by init_neural_network

TODO: It seems that the object function has converged to a minima. This is well supported by testing the network on test data.

In [ ]:
from test_network import test_trained_network

test_trained_network(
    network=network, x_test=x_test, y_test=y_test, num_ints=sort_params1.m
)

TODO: add comment on test

#### Scaling up the problem

We previously set the sequence length $r = 5$ and vocabulary size $m = 2$. The total amount of different sequences that can be generated is $2^5=32$. Since $2500$ sequences are created to train the neural network, there will most likely be extremely few or no new sequences to test the model on. Therefore the neural network will most likely be very accurate because it has been trained on every possible input it can receive.

Let us now train the network to sort sequences where $r = 7$ and $m = 5$. This will result in $5^7 = 78125$ possible input sequences and thus we expect the testing data to contain unknown sequences.

In [ ]:
from data_generators import get_train_test_sorting

training_data = get_train_test_sorting(
    length=sort_params2.r,
    num_ints=sort_params2.m,
    samples_per_batch=sort_params2.D,
    n_batches_train=sort_params2.b_train,
    n_batches_test=sort_params2.b_test,
)

x_train = training_data["x_train"]
y_train = training_data["y_train"][:, :, sort_params2.r - 1:]       # we slice y_train in order to get the correct size
x_test = training_data["x_test"]
y_test = training_data["y_test"]

In [ ]:
from train_network import train_network, init_neural_network
from layers_numba import CrossEntropy

network1 = init_neural_network(sort_params2)
network2 = init_neural_network(sort_params2)
network3 = init_neural_network(sort_params2)

networks = (network1, network2, network3)

loss = CrossEntropy()

L1, L2, L3 = [
    train_network(
    network=network,
    x_train=x_train,
    y_train=y_train,
    loss_func=loss,
    alpha=sort_params2.alpha,
    n_iter=sort_params2.n_iter,
    num_ints=sort_params2.m,
    dump_to_pickle_file=False,
) for network in networks
]

In [ ]:
plot_loss([L1, L2, L3], "Training for the sorting problem part 2", labels=("L1", "L2", "L3"))

In [ ]:
from test_network import test_trained_network

for n in networks:
    test_trained_network(
        network=n, x_test=x_test, y_test=y_test, num_ints=sort_params2.m
    )

TODO: https://datascience.stackexchange.com/questions/25024/strange-behavior-with-adam-optimizer-when-training-for-too-long

### 3.4: Addition problem

Let us now consider the problem of integer addition.
Given two two-digit non-negative integers we want to train our model to predict the sum going through the same procedure as for the problem of sorting integers.

Note that we now have $10^4$ possible input sequences: $a_0, a_1, b_0, b_1 \in \{0,1,2,\dots,9\}$.
The total number of training sequences are $ \text{num\_batches} \cdot \text{batch\_size} = 20 \cdot 250 = 5000$.
Therefore the training set will not contain all possibilities and our network will encounter unknown data during the testing phase.

In [ ]:
from data_generators import get_train_test_addition

training_data = get_train_test_addition(
    n_digit = add_params.r,
    samples_per_batch = add_params.D,
    n_batches_train = add_params.b_train,
    n_batches_test=add_params.b_test
)

x_train = training_data["x_train"]
y_train = training_data["y_train"][:, :, add_params.r*2 - 1:]
x_test = training_data["x_test"]
y_test = training_data["y_test"][:, :, ::-1]    # remember that (c0, c1, c2) is reversed in the training data.

In [ ]:
from train_network import train_network, init_neural_network
from layers_numba import CrossEntropy

network1 = init_neural_network(add_params)
network2 = init_neural_network(add_params)
network3 = init_neural_network(add_params)

networks = (network1, network2, network3)

loss = CrossEntropy()

L1, L2, L3 = [
    train_network(
        network=network,
        x_train=x_train,
        y_train=y_train,
        loss_func=loss,
        alpha=add_params.alpha,
        n_iter=add_params.n_iter,
        num_ints=add_params.m,
        dump_to_pickle_file=False
    ) for network in networks
]

In [ ]:
plot_loss([L1, L2, L3], "Training for the addition problem", ("net1", "net2", "net3"))

In [ ]:
from test_network import test_trained_network


for n in networks:
    test_trained_network(
        network=n, x_test=x_test, y_test=y_test, num_ints=add_params.m
    )

## Conclusion

The transformer model and Adam combine to create a neural network that predicts sequences of symbols.

We first saw rapid convergence of the loss function for the sorting problem with only zeros and ones.
The network then sorted correctly $100 \%$ of the test cases.
Trying to sort longer sequences of integers ranging from $0$ to $4$ also resultet in near $100\%$ score.

When adding integers the training time increased.
This is where `Numba` excels.
After compilation the training process is considerably faster than what we managed to do without `Numba`.

This is to be expected as the problem of addition is more involved than sorting.

---

## Bonus task: Text generation

The cells below are copied from the provided `text_generation`-notebook with a few additions.

In [ ]:
from train_test_params import TextGenParams

text_params = TextGenParams()

In [ ]:
from data_generators import text_to_training_data

with open('input.txt', 'r') as f:
    text = f.read()
    data,idx_to_text,text_to_idx, m = text_to_training_data(text_params.n_max,text,num_batches=text_params.b_train,batch_size=text_params.D)

    print("We will train on %d batches of size %d" % (len(data['x_train']),len(data['x_train'][0])))
    print("Each sequence has length %d" % text_params.n_max)

    print("Example of a sequence (chars): \n")
    print(''.join([idx_to_text[i] for i in data['x_train'][0][0]]))

    print("\nExample of a sequence (idx): \n")
    print(data['x_train'][0][0])

    text_params.m = m

In [ ]:
from train_network import init_neural_network

net = init_neural_network(text_params)

Training the network turns out to give (somewhat) decent results.

In [ ]:
import train_network
from layers_numba import CrossEntropy

loss = CrossEntropy()
train_network.train_network(
    network=net,
    x_train=np.array(data["x_train"]),
    y_train=np.array(data["y_train"]),
    loss_func=loss,
    alpha=text_params.alpha,
    n_iter=text_params.n_iter,
    num_ints=text_params.m,
    dump_to_pickle_file=True,
    file_name_dump="nn_text_gen.pkl",
)

In [ ]:
from text_generation import generate
import numpy as np

#We can now generate text from an initial string
start_text = "Thou shall not"
start_idx = np.array([text_to_idx[ch] for ch in start_text])[None]

#length of the total text sequence we want to generate
n_gen = 10*text_params.n_max

generated_idx = generate(net,start_idx,m,text_params.n_max,n_gen)

text = ''.join([idx_to_text[idx] for idx in generated_idx[0]])
print(text)

But we can do better.

First of all we gathered more training data. From [Project Gutenberg](https://www.gutenberg.org/) we found the complete works of Shakespeare.
It is found in the text file `spear.txt`.

Then we scaled up the training process. We first increased $L$ from $2$ to $3$.
After setting $\mathcal{D} = 200$ datapoints per batch and using $100$ batches the model was trained for approximately $15$ hours on the computation server Markov at IMF.
Only around $600$ iterations were done at this stage so we decreased the datapoints and batch size parameters. Then the model trained for approximately $9$ hours.
The dumped networks and the corresponding loss functions are found in `markov_dumps/`. Let's now generate some text with it!

In [ ]:
from data_generators import text_to_training_data

with open('shakespeare.txt', 'r') as f:
    text = f.read()
    data,idx_to_text,text_to_idx, m = text_to_training_data(text_params.n_max,text,num_batches=text_params.b_train,batch_size=text_params.D)

    print("We will train on %d batches of size %d" % (len(data['x_train']),len(data['x_train'][0])))
    print("Each sequence has length %d" % text_params.n_max)

    print("Example of a sequence (chars): \n")
    print(''.join([idx_to_text[i] for i in data['x_train'][0][0]]))

    print("\nExample of a sequence (idx): \n")
    print(data['x_train'][0][0])

    text_params.m = m

In [ ]:
from text_generation import generate
import numpy as np
from train_network import init_neural_network

markov_dump_dir = "markov_dumps/"

crazy_net1 = init_neural_network(text_params)
crazy_net1.load(markov_dump_dir + "nn_dump_text_gen_markov.pkl")

print("----------")
start_text = "Thou shall not"
start_idx = np.array([text_to_idx[ch] for ch in start_text])[None]

n_gen = 10*text_params.n_max
generated_idx = generate(crazy_net1,start_idx,m,text_params.n_max,n_gen)
text = ''.join([idx_to_text[idx] for idx in generated_idx[0]])
print(text)

print("\n----------")
start_text = "Nay, but speak not"
start_idx = np.array([text_to_idx[ch] for ch in start_text])[None]

n_gen = 10*text_params.n_max
generated_idx = generate(crazy_net1,start_idx,m,text_params.n_max,n_gen)
text = ''.join([idx_to_text[idx] for idx in generated_idx[0]])
print(text)
print("----------")

Plotting how the loss function evolved.

In [ ]:
L1 = np.load("markov_dumps/L_nn_dump_text_gen_markov_1.pkl", allow_pickle=True)
L2 = np.load("markov_dumps/L_nn_dump_text_gen_markov_2.pkl", allow_pickle=True)
plot_loss([ np.concatenate( (L1[L1>0], L2[L2 > 0]) ) ], "Loss function after training on Markov")

This network only trained for X iterations.
`FILENAM2` was trained for > iterations on SEERVERRR.
Let's plot the loss function and generate some text. 

In [ ]:
L = np.load("oracle_dumps/L_nn_dump_text_gen_markov_1.pkl", allow_pickle=True)
plot_loss([ L[L > 0] ], "Loss function after training on Oracle")

In [ ]:
from text_generation import generate
import numpy as np

text_params.m = 100   # 100 unique symbols in speare.txt
crazy_net2 = init_neural_network(text_params)
crazy_net2.load("FILENAME2")

print("----------")
start_text = "Thou shall not"
start_idx = np.array([text_to_idx[ch] for ch in start_text])[None]

n_gen = 10*text_params.n_max
generated_idx = generate(crazy_net2,start_idx,m,text_params.n_max,n_gen)
text = ''.join([idx_to_text[idx] for idx in generated_idx[0]])
print(text)

For comparison here we generate text *without* training the network...

In [ ]:
from text_generation import generate
import numpy as np
from train_network import init_neural_network

text_params.m = 65   # 65 unique symbols in input.txt
no_train_net = init_neural_network(text_params)

#We can now generate text from an initial string
start_text = "Thou shall not"
start_idx = np.array([text_to_idx[ch] for ch in start_text])[None]

#length of the total text sequence we want to generate
n_gen = 10*text_params.n_max

generated_idx = generate(no_train_net,start_idx,m,text_params.n_max,n_gen)

text = ''.join([idx_to_text[idx] for idx in generated_idx[0]])
print(text)

> And then the animator suffered a fatal heart attack 

[Monty Python](https://www.youtube.com/watch?v=3Q2WPneqhhs)